# 🔥 CNTEM'UP — Train Custom Bottle/Can Detector

This notebook trains a **YOLOv8n** model to detect bottles and cans,
then exports it to **ONNX** for browser use.

**Before running:**
1. Go to Runtime → Change runtime type → **T4 GPU**
2. Get your Roboflow API key from: app.roboflow.com → Settings → API Key
3. Run each cell in order

**Time:** ~1 hour total on free Colab T4

## Step 1: Install Dependencies

In [ ]:
!pip install ultralytics roboflow onnx onnxsim -q
print('✅ Dependencies installed!')

In [ ]:
# Verify GPU is available
import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "❌ NO GPU — go to Runtime → Change runtime type → T4 GPU"}')
print(f'CUDA: {torch.cuda.is_available()}')

## Step 2: Download Dataset from Roboflow

**Option A:** Use the public "Bottle and Can" dataset (1,200 images, 2 classes)

**Option B:** Use your own Roboflow project (if you uploaded + labeled your own images)

Fill in your API key below and choose your option.

In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️ FILL THESE IN:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"  # Get from app.roboflow.com → Settings

# Option A: Public "Bottle and Can" dataset (recommended to start)
WORKSPACE = "bouteille"
PROJECT = "bottle-and-can"
VERSION = 1

# Option B: Your own project — uncomment and fill in:
# WORKSPACE = "your-workspace-name"
# PROJECT = "your-project-name"
# VERSION = 1

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
version = project.version(VERSION)
dataset = version.download("yolov8")

dataset_path = dataset.location
print(f'\n✅ Dataset downloaded to: {dataset_path}')

# Show what we got
import os
for split in ['train', 'valid', 'test']:
    img_dir = os.path.join(dataset_path, split, 'images')
    if os.path.exists(img_dir):
        count = len(os.listdir(img_dir))
        print(f'  {split}: {count} images')

## Step 3: Check Class Balance

Make sure bottle and can have roughly equal representation.
If one class has 5x more than the other, accuracy will suffer.

In [ ]:
from collections import Counter
import os

class_counts = Counter()
lbl_dir = os.path.join(dataset_path, 'train', 'labels')

for fname in os.listdir(lbl_dir):
    if fname.endswith('.txt'):
        with open(os.path.join(lbl_dir, fname)) as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    class_counts[int(parts[0])] += 1

# Read class names from data.yaml
import yaml
with open(os.path.join(dataset_path, 'data.yaml')) as f:
    data_yaml = yaml.safe_load(f)

names = data_yaml.get('names', {})
if isinstance(names, list):
    names = {i: n for i, n in enumerate(names)}

print('📊 Class distribution (training set):')
for cls_id, count in sorted(class_counts.items()):
    name = names.get(cls_id, f'class_{cls_id}')
    print(f'  {name} (id={cls_id}): {count} annotations')

total = sum(class_counts.values())
print(f'\n  Total: {total} annotations')

## Step 4: Visualize Some Training Images

Sanity check — make sure the labels look right.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random

colors = ['#10b981', '#ef4444', '#3b82f6', '#f59e0b']

img_dir = os.path.join(dataset_path, 'train', 'images')
lbl_dir = os.path.join(dataset_path, 'train', 'labels')
img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
samples = random.sample(img_files, min(6, len(img_files)))

for ax, fname in zip(axes.flat, samples):
    img = Image.open(os.path.join(img_dir, fname))
    w, h = img.size
    ax.imshow(img)
    ax.set_title(fname[:20], fontsize=8)
    ax.axis('off')

    # Draw labels
    lbl_file = os.path.splitext(fname)[0] + '.txt'
    lbl_path = os.path.join(lbl_dir, lbl_file)
    if os.path.exists(lbl_path):
        with open(lbl_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls_id = int(parts[0])
                    cx, cy, bw, bh = map(float, parts[1:5])
                    x1 = (cx - bw/2) * w
                    y1 = (cy - bh/2) * h
                    box_w = bw * w
                    box_h = bh * h
                    color = colors[cls_id % len(colors)]
                    rect = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor='none')
                    ax.add_patch(rect)
                    label = names.get(cls_id, f'cls_{cls_id}')
                    ax.text(x1, y1-5, label, color=color, fontsize=8, fontweight='bold')

plt.tight_layout()
plt.suptitle('Training Samples with Labels', fontsize=14, y=1.02)
plt.show()

## Step 5: TRAIN 🚀

This is where the magic happens. Takes ~30-90 min on a T4 GPU.

**Key settings:**
- `epochs=100` — model will early-stop if it plateaus
- `imgsz=640` — standard YOLO input (matches our React hook)
- `batch=16` — sweet spot for free Colab T4 (16GB VRAM)
- Starting from pre-trained COCO weights (transfer learning = way faster + better)

In [ ]:
from ultralytics import YOLO

# Start from COCO pre-trained weights (transfer learning)
model = YOLO('yolov8n.pt')  # nano = smallest, fastest

# Train!
results = model.train(
    data=os.path.join(dataset_path, 'data.yaml'),
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,        # Early stop if no improvement for 20 epochs
    device=0,           # GPU
    workers=2,          # Colab has limited CPU
    project='cntemup',  # Output folder
    name='bottle_can',  # Run name
    
    # Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
)

print('\n✅ Training complete!')

## Step 6: Check Results

**What to look for:**
- `mAP50 > 0.80` = good
- `mAP50 > 0.90` = excellent
- `mAP50 < 0.60` = needs more/better training data

In [ ]:
from IPython.display import Image, display

# Training curves
print('📈 Training Results:')
display(Image(filename='cntemup/bottle_can/results.png', width=900))

# Confusion matrix
print('\n🔄 Confusion Matrix:')
display(Image(filename='cntemup/bottle_can/confusion_matrix.png', width=600))

In [ ]:
# Validation predictions — see how the model performs
print('🎯 Validation Predictions:')
try:
    display(Image(filename='cntemup/bottle_can/val_batch0_pred.jpg', width=900))
except:
    print('(validation images not generated — this is fine)')

## Step 7: Test on Sample Images

Run the model on validation images to see it in action.

In [ ]:
# Load the best model from training
best_model = YOLO('cntemup/bottle_can/weights/best.pt')

# Run on validation images
val_imgs = os.path.join(dataset_path, 'valid', 'images')
test_results = best_model.predict(
    source=val_imgs,
    save=True,
    conf=0.35,
    project='cntemup',
    name='test_predictions',
)

# Show some results
pred_dir = 'cntemup/test_predictions'
pred_files = [f for f in os.listdir(pred_dir) if f.endswith(('.jpg', '.png'))]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
for ax, fname in zip(axes.flat, pred_files[:6]):
    img = Image.open(os.path.join(pred_dir, fname))
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.suptitle('Model Predictions', fontsize=14, y=1.02)
plt.show()

# Print metrics
metrics = best_model.val(data=os.path.join(dataset_path, 'data.yaml'))
print(f'\n📊 Final Metrics:')
print(f'  mAP50:    {metrics.box.map50:.3f}')
print(f'  mAP50-95: {metrics.box.map:.3f}')
print(f'  Precision: {metrics.box.mp:.3f}')
print(f'  Recall:    {metrics.box.mr:.3f}')

## Step 8: Export to ONNX

This converts the model to a format that runs in the browser via ONNX Runtime Web.

In [ ]:
# Export to ONNX for browser
best_model = YOLO('cntemup/bottle_can/weights/best.pt')

onnx_path = best_model.export(
    format='onnx',
    opset=12,        # Browser-compatible
    simplify=True,   # Smaller file, faster inference
    imgsz=640,       # Must match INPUT_SIZE in React hook
)

import os
size_mb = os.path.getsize(onnx_path) / (1024 * 1024)
print(f'\n✅ Exported to: {onnx_path}')
print(f'📦 Model size: {size_mb:.1f} MB')
print(f'\nNext: Download this file and put it at:')
print(f'  app/public/models/bottle-can.onnx')

## Step 9: Download the Model

Click the download button or run this cell.

Then put the file at: **`app/public/models/bottle-can.onnx`** in your CNTEM'UP project.

In [ ]:
from google.colab import files

# Download the ONNX model
files.download('cntemup/bottle_can/weights/best.onnx')

print('\n🎉 Done! Put this file at:')
print('   app/public/models/bottle-can.onnx')
print('\nThe app will auto-detect it and switch from COCO-SSD to your custom model.')

## 🎉 You're done!

**What just happened:**
1. Downloaded a labeled bottle/can dataset
2. Trained YOLOv8n on it (transfer learning from COCO)
3. Exported to ONNX for browser use

**Next steps:**
1. Put `best.onnx` → `app/public/models/bottle-can.onnx`
2. Run `npm run dev` — the app auto-detects the model
3. Footer will show "YOLOv8 Custom engine"
4. Test and tune `CONF_THRESHOLD` in `useYoloDetection.js`

**Want better accuracy?**
- Add your own photos to the Roboflow project
- Re-generate the dataset version
- Re-run this notebook
- Each iteration gets better!